In [2]:
import os
import tavily
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
# To install: pip install tavily-python
from tavily import TavilyClient
from IPython.display import display_markdown


In [3]:

load_dotenv()
client = TavilyClient(os.getenv("TAVILY_API_KEY"))


In [4]:
from typing import TypedDict

class AgentState(TypedDict):
    question: str
    search_results: dict  
    answer: str

In [5]:
def search_web(state: AgentState):
    """
    Node 1: Intelligent Web Searcha

    Uses Tavily's AI-optimized search to find and process web information.
    Behind the scenes: Tavily searches multiple sources, extracts relevant content,
    and uses AI to synthesize the information into a coherent answer.
    """
    #print(f"🔍 Searching: {state['question']}")
    client = TavilyClient(os.getenv("TAVILY_API_KEY"))
    
    
    search_results = client.search(
        query=state["question"],
        max_results=3,        
        include_answer=True      
    )

    return {"search_results": search_results}

In [6]:
def generate_answer(state: AgentState):
    """
    Node 2: Answer Synthesis and Formatting

    Takes the search results from Tavily and formats them into a clean,
    user-friendly response with proper source attribution.
    """
    #print("🤖 Formatting answer...")

    # Extract Tavily's AI-generated answer (the smart synthesis)
    ai_answer = state["search_results"].get("answer", "No answer found")

    # Extract source URLs for transparency and verification
    sources = [f"- {result['title']}: {result['url']}" 
              for result in state["search_results"]["results"]]

    # Combine the intelligent answer with source attribution
    final_answer = f"{ai_answer}\n\nSources:\n" + "\n".join(sources)

    return {"answer": final_answer}

In [ ]:
def create_agent():
    """
    Build the AI Agent Workflow

    This creates a StateGraph where:
    - Each node is an independent function that can read/update shared state
    - LangGraph handles orchestration, state management, and execution flow
    - Easy to extend: just add more nodes and define their connections
    """
    # Create the workflow graph
    workflow = StateGraph(AgentState)

    # Define our processing nodes
    workflow.add_node("search", search_web)        # Step 1: Gather information  
    workflow.add_node("result", generate_answer)   # Step 2: Process and format

    # Define the flow of intelligence
    workflow.set_entry_point("search")      # Start here
    workflow.add_edge("search", "result")   # After search, go to answer
    workflow.add_edge("result", END)        # After answer, we're done

    return workflow.compile()

In [13]:
agent = create_agent()

In [14]:
question = "Everything about the red fort blast"
output = agent.invoke({"question": question})
display_markdown(output["answer"], raw=True)

On November 10, 2025, a car exploded near Delhi's Red Fort, killing at least 13 people. Indian police are investigating under anti-terrorism laws. The incident involved trained doctors as suspects.

Sources:
- Delhi Red Fort Blast LIVE Updates Car Explosion - NDTV: https://www.ndtv.com/india-news/delhi-red-fort-blast-live-updates-car-explosion-many-feared-dead-after-explosion-in-car-near-red-fort-in-delhi-pm-modi-rekha-gupta-delhi-9609750
- Doctors Behind Delhi Blast, White Collar Terrorism on the Rise ...: https://www.youtube.com/watch?v=R_9s6AIYoeg
- Delhi Red Fort blast kills 13: What happened as police invoke 'terror ...: https://www.aljazeera.com/news/2025/11/11/delhi-red-fort-blast-kills-13-what-happened-as-police-invoke-terror-law